# Лабораторная работа №5. Линейная регрессия

## Описание задачи и загрузка данных

Импорт библиотек

В этой работе будем работать с одним из классических наборов данных в статистике, содержащим информацию о бриллиантах и решать задачу предсказания цены бриллианта price в зависимости от его характеристик.. Описание можно посмотреть здесь (https://www.kaggle.com/datasets/shivam2503/diamonds). Загрузите датасет `diamonds.csv`

In [4]:
# Импорт библиотек
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
df = pd.read_csv("data/diamonds.csv")
df.head()

,Unnamed: 0,carat,cut,color,clarity,depth,table,price,x,y,z
0,1,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,2,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,3,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,4,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,5,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75


## Построение модели

Есть ли в наборе данных пропущенные значения? Если да, удалите их. Также выведите на экран число пропусков в каждом столбце.

In [6]:
print(df.isnull().sum())

print(f"\nNumber of duplicate rows: {df.duplicated().sum()}")

df.dropna(inplace=True)

Unnamed: 0    0
carat         0
cut           0
color         0
clarity       0
depth         0
table         0
price         0
x             0
y             0
z             0
dtype: int64

Number of duplicate rows: 0


In [7]:
df = df.drop_duplicates()
print(f"\nNumber of duplicate rows after removing: {df.duplicated().sum()}")


Number of duplicate rows after removing: 0


Есть ли в наборе данных бессмысленные столбцы (признаки, не несущие дополнительной информации)? Если да, то удалите их.

In [8]:
df = df.drop("Unnamed: 0", axis=1)
# !             ^^^-------------------Дублирует индексы

Линейная регрессия основана на предположении о линейной связи между признаками и целевой переменной, а потому перед выбором переменных для включения в модель имеет смысл проверить, насколько эта связь выполняется. Выясним корреляцию между признаками.
Выведите матрицу корреляций между всеми вещественными признаками и целевой переменной.
Какой вещественный признак коррелирует с целевой переменной больше всего?

In [9]:
# Словарь для замены значений
replace_temp = {
    'cut': {'Fair': 0, 'Good': 1, 'Very Good': 2, 'Premium': 3, 'Ideal': 4},
    'color': {'D': 6, 'E': 5, 'F': 4, 'G': 3, 'H': 2, 'I': 1, 'J': 0},
    'clarity': {'I1': 0, 'SI2': 1, 'SI1': 2, 'VS2': 3, 'VS1': 4, 'VVS2': 5, 'VVS1': 6, 'IF': 7}
}

# Замена значений непосредственно в df
for column, mapping in replace_temp.items():
    df[column] = df[column].replace(mapping)

# Проверка результата
print(df)

       carat  cut  color  clarity  depth  table  price     x     y     z
0       0.23    4      5        1   61.5   55.0    326  3.95  3.98  2.43
1       0.21    3      5        2   59.8   61.0    326  3.89  3.84  2.31
2       0.23    1      5        4   56.9   65.0    327  4.05  4.07  2.31
3       0.29    3      1        3   62.4   58.0    334  4.20  4.23  2.63
4       0.31    1      0        1   63.3   58.0    335  4.34  4.35  2.75
...      ...  ...    ...      ...    ...    ...    ...   ...   ...   ...
53935   0.72    4      6        2   60.8   57.0   2757  5.75  5.76  3.50
53936   0.72    1      6        2   63.1   55.0   2757  5.69  5.75  3.61
53937   0.70    2      6        2   62.8   60.0   2757  5.66  5.68  3.56
53938   0.86    3      2        1   61.0   58.0   2757  6.15  6.12  3.74
53939   0.75    4      6        1   62.2   55.0   2757  5.83  5.87  3.64

[53940 rows x 10 columns]


C:\Users\vlad9\AppData\Local\Temp\ipykernel_9996\3483754725.py:10: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[column] = df[column].replace(mapping)


In [10]:
correlation_matrix = df.corr()
print(correlation_matrix)

            carat       cut     color   clarity     depth     table     price  \
carat    1.000000 -0.134967 -0.291437 -0.352841  0.028224  0.181618  0.921591   
cut     -0.134967  1.000000  0.020519  0.189175 -0.218055 -0.433405 -0.053491   
color   -0.291437  0.020519  1.000000 -0.025631 -0.047279 -0.026465 -0.172511   
clarity -0.352841  0.189175 -0.025631  1.000000 -0.067384 -0.160327 -0.146800   
depth    0.028224 -0.218055 -0.047279 -0.067384  1.000000 -0.295779 -0.010647   
table    0.181618 -0.433405 -0.026465 -0.160327 -0.295779  1.000000  0.127134   
price    0.921591 -0.053491 -0.172511 -0.146800 -0.010647  0.127134  1.000000   
x        0.975094 -0.125565 -0.270287 -0.371999 -0.025289  0.195344  0.884435   
y        0.951722 -0.121462 -0.263584 -0.358420 -0.029341  0.183760  0.865421   
z        0.953387 -0.149323 -0.268227 -0.366952  0.094924  0.150929  0.861249   

                x         y         z  
carat    0.975094  0.951722  0.953387  
cut     -0.125565 -0.121462 

In [11]:
# Выяснение признака с наибольшей корреляцией с price
correlation_with_price = correlation_matrix['price'].sort_values(ascending=False)
print("\nНаиболее коррелирующий признак с price:\n", correlation_with_price[1:2])


Наиболее коррелирующий признак с price:
 carat    0.921591
Name: price, dtype: float64


- Наиболее коррелирующий с price признак — это `carat` (коэффициент корреляции **0.92**). Это говорит о том, что вес бриллианта (карат) наиболее тесно связан с его ценой, что логично, поскольку больший вес, как правило, увеличивает стоимость бриллианта.
- Другие числовые признаки, такие как `x`, `y`, и `z` (размеры бриллианта), также показывают высокую корреляцию с price (в пределах **0.86**–**0.88**), так как они связаны с размером и массой бриллианта.
- Признаки `cut`, `color`, и `clarity` показывают меньшую корреляцию с `price`. В отличие от веса и размеров, они имеют косвенное влияние на цену, возможно, из-за их дискретного характера.

Так как линейная модель складывает значения признаков с некоторыми весами, нам нужно аккуратно обработать категориальные признаки. Закодируйте категориальные переменные при помощи OneHot-кодирования (pd.get_dummies). Не забудьте поставить значение параметра drop_first равным True.

P.S. Числовые столбцы оставляем в таблице без изменений.

In [12]:
# OneHot-кодирование категориальных признаков
df = pd.get_dummies(df, drop_first=True)

In [13]:
print(df)

       carat  cut  color  clarity  depth  table  price     x     y     z
0       0.23    4      5        1   61.5   55.0    326  3.95  3.98  2.43
1       0.21    3      5        2   59.8   61.0    326  3.89  3.84  2.31
2       0.23    1      5        4   56.9   65.0    327  4.05  4.07  2.31
3       0.29    3      1        3   62.4   58.0    334  4.20  4.23  2.63
4       0.31    1      0        1   63.3   58.0    335  4.34  4.35  2.75
...      ...  ...    ...      ...    ...    ...    ...   ...   ...   ...
53935   0.72    4      6        2   60.8   57.0   2757  5.75  5.76  3.50
53936   0.72    1      6        2   63.1   55.0   2757  5.69  5.75  3.61
53937   0.70    2      6        2   62.8   60.0   2757  5.66  5.68  3.56
53938   0.86    3      2        1   61.0   58.0   2757  6.15  6.12  3.74
53939   0.75    4      6        1   62.2   55.0   2757  5.83  5.87  3.64

[53940 rows x 10 columns]


Создайте матрицу X, содержащую все признаки, и не содержащую целевую переменную price. Также создайте вектор y, содержащий целевую переменную price.

In [14]:
# Создание матрицы X, содержащей все признаки, кроме целевой переменной price
X = df.drop(columns=['price'])

# Создание вектора y, содержащего целевую переменную price
y = df['price']

# Вывод результатов
print("Матрица X:")
print(X)
print("\nВектор y:")
print(y)

Матрица X:
       carat  cut  color  clarity  depth  table     x     y     z
0       0.23    4      5        1   61.5   55.0  3.95  3.98  2.43
1       0.21    3      5        2   59.8   61.0  3.89  3.84  2.31
2       0.23    1      5        4   56.9   65.0  4.05  4.07  2.31
3       0.29    3      1        3   62.4   58.0  4.20  4.23  2.63
4       0.31    1      0        1   63.3   58.0  4.34  4.35  2.75
...      ...  ...    ...      ...    ...    ...   ...   ...   ...
53935   0.72    4      6        2   60.8   57.0  5.75  5.76  3.50
53936   0.72    1      6        2   63.1   55.0  5.69  5.75  3.61
53937   0.70    2      6        2   62.8   60.0  5.66  5.68  3.56
53938   0.86    3      2        1   61.0   58.0  6.15  6.12  3.74
53939   0.75    4      6        1   62.2   55.0  5.83  5.87  3.64

[53940 rows x 9 columns]

Вектор y:
0         326
1         326
2         327
3         334
4         335
         ... 
53935    2757
53936    2757
53937    2757
53938    2757
53939    2757
Name: 

Разделите выборку на тренировочную и тестовую. Долю тестовой выборки укажите равной 0.3

In [15]:
from sklearn.model_selection import train_test_split

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

Масштабируйте вещественные признаки тренировочной и тестовой выборок при помощи модуля StandardScaler.

In [17]:
from sklearn.preprocessing import StandardScaler

In [18]:
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)

X_test_scaled = scaler.transform(X_test)

Обучите модель линейной регрессии

Выведите на экран веса, которые линейная регрессия присвоила признакам

In [19]:
from sklearn.linear_model import LinearRegression

In [20]:
model = LinearRegression()
model.fit(X_train, y_train)

# Вывод весов
weights = model.coef_
print("Веса признаков:\n", weights)

Веса признаков:
 [10697.32682679   120.5213228    324.12780762   504.09390002
   -78.64608396   -27.50801659  -842.42154479    25.3297773
   -23.67989821]


- Самые значимые положительные веса при обучении линейной регрессии получила переменная `carat`, что подтверждает выводы корреляционного анализа. Она оказывает наибольшее влияние на итоговую стоимость.
- Категориальные признаки (`cut`, `color`, и `clarity`) получили меньшие веса, что указывает на их менее выраженное влияние на цену в сравнении с весом (`carat`) и размерами (`x`, `y`, `z`).
- Признак `depth` получил отрицательное значение, что может указывать на негативное влияние на цену при увеличении значения глубины. Показатель глубины, вероятно, влияет на оптические характеристики и может быть менее важным фактором, чем другие параметры

Оцените качество модели

In [21]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [22]:
y_pred = model.predict(X_test)

r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print(f"Качество модели:\nR^2: {r2:.4f}\nMAE: {mae:.2f}\nRMSE: {rmse:.2f}")

Качество модели:
R^2: 0.9075
MAE: 802.20
RMSE: 1201.21


Коэффициент детерминации 
$𝑅^2=0.9075$ говорит о том, что модель объясняет около **90.75%** вариации цены бриллиантов. Это достаточно высокий показатель для линейной модели, что говорит о хорошей пригодности признаков для предсказания цены.

## Попытка улучшить качество модели

Как можно заметить из анализа корреляционной матрицы, между некоторыми признаками имеется сильная корреляция, что может быть индикатором проблемы мультиколлинеарности. Для решения этой проблемы можно либо исключить некоторые признаки из модели (например, если признак линейно зависим с какими-то другими, его можно исключить из модели, т.е. удалить из матрицы объект-признак и заново обучить модель).

Удалите из матриц Xtrain и Xtest признак, который наиболее сильно коррелирует с остальными. Заново обучите модель и оцените её качество. Улучшилось ли качество модели?

Попробуйте удалить какой-то другой признак (можете попробовать несколько вариантов). Помогло ли это улучшить качество модели?

In [23]:
# копируем
X_train_copy = X_train.copy()
X_test_copy = X_test.copy()

def evaluate_model(X_train, X_test, y_train, y_test):
    model = LinearRegression()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    return r2, mae, rmse

# Удаляем признак 'x' и обучаем модель
X_train_x_dropped = X_train_copy.drop(columns=['x'])
X_test_x_dropped = X_test_copy.drop(columns=['x'])
r2_x, mae_x, rmse_x = evaluate_model(X_train_x_dropped, X_test_x_dropped, y_train, y_test)

# Удаляем признак 'y' и обучаем модель
X_train_y_dropped = X_train_copy.drop(columns=['y'])
X_test_y_dropped = X_test_copy.drop(columns=['y'])
r2_y, mae_y, rmse_y = evaluate_model(X_train_y_dropped, X_test_y_dropped, y_train, y_test)

# Удаляем признак 'z' и обучаем модель
X_train_z_dropped = X_train_copy.drop(columns=['z'])
X_test_z_dropped = X_test_copy.drop(columns=['z'])
r2_z, mae_z, rmse_z = evaluate_model(X_train_z_dropped, X_test_z_dropped, y_train, y_test)

# Результаты
print("Results with 'x' dropped: R^2 =", r2_x, "MAE =", mae_x, "RMSE =", rmse_x)
print("Results with 'y' dropped: R^2 =", r2_y, "MAE =", mae_y, "RMSE =", rmse_y)
print("Results with 'z' dropped: R^2 =", r2_z, "MAE =", mae_z, "RMSE =", rmse_z)

c:\Users\vlad9\Documents\study\4 kurs\1 semestr\mmo\.venv\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\vlad9\Documents\study\4 kurs\1 semestr\mmo\.venv\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Results with 'x' dropped: R^2 = 0.9064396809771065 MAE = 820.8829146645131 RMSE = 1207.9583217743657
Results with 'y' dropped: R^2 = 0.9074680832173811 MAE = 802.1746080128538 RMSE = 1201.3011210718282
Results with 'z' dropped: R^2 = 0.9074812143411896 MAE = 802.22214336385 RMSE = 1201.2158802491251


c:\Users\vlad9\Documents\study\4 kurs\1 semestr\mmo\.venv\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Иногда генерация новых признаков помогает модели лучше находить взаимосвязи между целевой переменной и признаками. Попробуйте придумать новые признаки и добавить их в модель.

Помогло ли это улучшить качество модели?

In [30]:
# Обработка NaN значений в новых признаках
X_train_copy['xy_ratio'].fillna(X_train_copy['xy_ratio'].mean(), inplace=True)
X_test_copy['xy_ratio'].fillna(X_test_copy['xy_ratio'].mean(), inplace=True)

X_train_copy['depth_table_ratio'].fillna(X_train_copy['depth_table_ratio'].mean(), inplace=True)
X_test_copy['depth_table_ratio'].fillna(X_test_copy['depth_table_ratio'].mean(), inplace=True)

# Проверка и обучение модели
r2_new, mae_new, rmse_new = evaluate_model(X_train_copy, X_test_copy, y_train, y_test)

# Вывод результатов
print("Results with new features: R^2 =", r2_new, "MAE =", mae_new, "RMSE =", rmse_new)


Results with new features: R^2 = 0.9073839332975575 MAE = 793.9933766393364 RMSE = 1201.8472376115399


C:\Users\vlad9\AppData\Local\Temp\ipykernel_9996\609159599.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_train_copy['xy_ratio'].fillna(X_train_copy['xy_ratio'].mean(), inplace=True)
C:\Users\vlad9\AppData\Local\Temp\ipykernel_9996\609159599.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always b

Попробуйте улучшить модель, используя кросс-валидацию и регуляризацию

In [25]:
from sklearn.linear_model import Ridge, Lasso
from sklearn.model_selection import cross_val_score, GridSearchCV

In [27]:
# ! 1. Подбор оптимального alpha для Ridge
ridge_alphas = np.logspace(-2, 3, 100)  # диапазон значений alpha для Ridge
ridge_scores = []

for alpha in ridge_alphas:
    ridge_model = Ridge(alpha=alpha)
    score = cross_val_score(ridge_model, X_train_scaled, y_train, cv=5, scoring='r2').mean()
    ridge_scores.append(score)

best_alpha_ridge = ridge_alphas[np.argmax(ridge_scores)]
print("Лучший alpha для Ridge:", best_alpha_ridge)

# 2. Подбор оптимального alpha для Lasso, начиная с найденного alpha для Ridge
lasso_alphas = np.linspace(0.1 * best_alpha_ridge, 2 * best_alpha_ridge, 50)  # диапазон alpha на основе Ridge
lasso_scores = []

for alpha in lasso_alphas:
    lasso_model = Lasso(alpha=alpha, max_iter=10000)  # увеличим max_iter для лучшей сходимости
    score = cross_val_score(lasso_model, X_train_scaled, y_train, cv=5, scoring='r2').mean()
    lasso_scores.append(score)

best_alpha_lasso = lasso_alphas[np.argmax(lasso_scores)]
print("Лучший alpha для Lasso:", best_alpha_lasso)

# Оценка моделей с наилучшими значениями alpha
ridge_model_final = Ridge(alpha=best_alpha_ridge).fit(X_train_scaled, y_train)
lasso_model_final = Lasso(alpha=best_alpha_lasso, max_iter=10000).fit(X_train_scaled, y_train)

ridge_score_final = ridge_model_final.score(X_test_scaled, y_test)
lasso_score_final = lasso_model_final.score(X_test_scaled, y_test)

print("Оценка Ridge модели:", ridge_score_final)
print("Оценка Lasso модели:", lasso_score_final)

Лучший alpha для Ridge: 61.35907273413169
Лучший alpha для Lasso: 6.13590727341317
Оценка Ridge модели: 0.9072794712275332
Оценка Lasso модели: 0.9071261947331591


1. **Удаление коррелирующих признаков** немного улучшило `MAE`, особенно после удаления признака `y`. Однако общая метрика $R^2$ осталась практически без изменений. Это свидетельствует о том, что мультиколлинеарность не сильно мешала модели.

2. **Добавление новых признаков** слегка улучшило метрику `MAE`, но практически не повлияло на $R^2$ и `RMSE`. Видимо, добавленные признаки имеют небольшое значение для модели.

3. **Кросс-валидация и регуляризация** (*Ridge* и *Lasso*): Использование кросс-валидации и регуляризации с подбором `alpha` также не привело к значительным улучшениям. *Ridge* показала немного лучшую оценку $R^2$, но разница минимальна.

### Рекомендации
- **Комбинированная регуляризация** (`Elastic Net`): Можно попробовать `Elastic Net`, чтобы объединить преимущества `L1` и `L2` регуляризации. Он может учесть взаимосвязи и контролировать мультиколлинеарность.

- **Нелинейные модели**: Возможно, стоит попробовать нелинейные модели (например, `Random Forest` или `Gradient Boosting`), если линейные методы исчерпали себя.